In [1]:
%load_ext autoreload
%autoreload 2
standard=True
standard=False
bureau = False
#  bureau = True
" Interest_Rate "
import gc
import numpy as np
import pandas as pd
import sys
import re
from glob import glob
import os
HOME = os.path.expanduser('~')
sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
#  sys.path.append(f"/mnt/c/Git/go/kaggle/github/library/")
import utils
from utils import logger_func, mkdir_func

#========================================================================
# Global Variable
sys.path.append('../py')
from info_home_credit import hcdr_key_cols
key, target, ignore_list = hcdr_key_cols()
#========================================================================

prev_key = 'SK_ID_PREV'
acr = 'AMT_CREDIT'
aan = 'AMT_ANNUITY'
adp = 'AMT_DOWN_PAYMENT'
cpy = 'CNT_PAYMENT'
co_type = 'NAME_CONTRACT_TYPE'
dd = 'DAYS_DECISION'

In [15]:
========================================================================
Previous ApplicationのInterest Rateを計算
base = utils.read_df_pkl('../input/base0*').set_index(key)
app = utils.read_df_pkl('../input/clean_app*')[[key, target]]
df = utils.read_df_pkl('../input/clean_prev*')
df = df[[key, prev_key, dd, acr, aan, cpy, adp, co_type]].merge(app, on=key, how='inner')
df = df[~df[cpy].isnull()]
cpy = 'CNT_PAYMENT'
df['Pred_CNT_PAYMENT'] = utils.read_pkl_gzip('../features/other/previous_pred_CNT_PAYMENT_CV2-9225943085922026.gz')
df.loc[df[cpy].isnull(), cpy] = df.loc[df[cpy].isnull(), 'Pred_CNT_PAYMENT']

for cnt in range(3, 64, 3):
    if cnt<=60:
        ir = ( (df[aan].values * cnt) / df[acr].values ) - 1.0
        df[f'ir_{cnt}'] = ir
        df[f'ir_{cnt}'] = df[f'ir_{cnt}'].map(lambda x: x if (0.08<x) and (x<0.5) else np.nan)
        print(f"{cnt} :", len(df[f'ir_{cnt}'].dropna()))
        if len(df[f'ir_{cnt}'].dropna())<len(df)*0.001:
            df.drop(f'ir_{cnt}', axis=1, inplace=True)
            continue
    else:
        ir = ( (df[aan].values * df[cpy].values) / df[acr].values ) - 1.0
        df[f'ir_pred'] = ir
        df[f'ir_pred'] = df[f'ir_pred'].map(lambda x: x if (0.08<=x) and (x<=0.5) else np.nan)
        cnt = 'pred'

ir_cols = [col for col in df.columns if col.count('ir_')]

aggs = {}

for col in ir_cols:
    aggs[col] = ['mean', 'max', 'min', 'std']

df_agg = df[[key]+ir_cols].groupby(key).agg(aggs)
new_cols = [f"{k}-{method}" for k in aggs.keys() for method in aggs[k]]
df_agg.columns = new_cols
df_agg = base.join(df_agg)

prefix = 'f006_pre_'
utils.save_feature(df_feat=df_agg, ignore_list=ignore_list, is_train=2, target=target, prefix=prefix)

del df, df_agg
gc.collect()

100%|██████████| 3/3 [00:00<00:00,  4.08it/s]


(307511,) | ir_pred-mean
(307511,) | ir_pred-max
(307511,) | ir_pred-min
(307511,) | ir_pred-std
(48744,) | ir_pred-mean
(48744,) | ir_pred-max
(48744,) | ir_pred-min
(48744,) | ir_pred-std


77

In [17]:
# Current Applicationに対するCNT_PAYMENTの予測値
trn_cpy = utils.read_pkl_gzip('../features/train_f005_app_App_Pred_CNT_PAYMENT.gz')
tes_cpy = utils.read_pkl_gzip('../features/test_f005_app_App_Pred_CNT_PAYMENT.gz')
pred_cpy = np.hstack((trn_cpy, tes_cpy))
df = utils.read_df_pkl('../input/clean_app*')
df[cpy] = pred_cpy

#========================================================================
# Current ApplicationのInterest Rateを計算
# 金利が何回分の支払いに対して発生しているか不明なので、3回刻みで一通り作る
for cnt in range(9, 40, 3):
#  for cnt in range(27, 46, 3):
    if cnt%6!=3 and cnt%12!=-3:
        continue
    if cnt<=60:
    #  if cnt<=45:
        ir = ( (df[aan].values * cnt) / df[acr].values ) - 1.0
        df[f'ir_{cnt}'] = ir
        #  if cnt==9 or cnt==15 or cnt==21 or cnt==27 or cnt==33:
        if cnt==6:
            pass
        #  elif np.abs(cnt%12)==3:
        #  elif cnt%6==3 or cnt%12==-3:
        #      df[f'ir_{cnt}'] = df[f'ir_{cnt}'].map(lambda x: x if (0.08<=x) and (x<=0.25) else np.nan)
        elif cnt==21:
            df[f'ir_{cnt}'] = df[f'ir_{cnt}'].map(lambda x: x if (0.08<=x) and (x<=0.23) else np.nan)
        elif cnt==27:
            df[f'ir_{cnt}'] = df[f'ir_{cnt}'].map(lambda x: x if (0.08<=x) and (x<=0.23) else np.nan)
        elif cnt==33:
            df[f'ir_{cnt}'] = df[f'ir_{cnt}'].map(lambda x: x if (0.08<=x) and (x<=0.23) else np.nan)
        else:
            df[f'ir_{cnt}'] = df[f'ir_{cnt}'].map(lambda x: x if (0.08<=x) and (x<=0.5) else np.nan)
        print(f"{cnt} :", len(df[f'ir_{cnt}'].dropna()))
        if len(df[f'ir_{cnt}'].dropna())<len(df)*0.001:
            df.drop(f'ir_{cnt}', axis=1, inplace=True)
            continue
    else:
        ir = ( (df[aan].values * df[cpy].values) / df[acr].values ) - 1.0
        df[f'ir_pred'] = ir
        df[f'ir_pred'] = df[f'ir_pred'].map(lambda x: x if (0.08<=x) and (x<=0.5) else np.nan)
        cnt = 'pred'

ir_cols = [col for col in df.columns if col.count('ir_')]
df.set_index(key, inplace=True)
df = base.join(df[ir_cols])

prefix = 'f006_app_'
utils.save_feature(df_feat=df, ignore_list=ignore_list, is_train=2, target=target, prefix=prefix)

100%|██████████| 3/3 [00:00<00:00, 10.46it/s]


9 : 595
15 : 48506
21 : 35775
27 : 29346
33 : 22074
39 : 79744
(307511,) | ir_9
(307511,) | ir_15
(307511,) | ir_21
(307511,) | ir_27
(307511,) | ir_33
(307511,) | ir_39
(48744,) | ir_9
(48744,) | ir_15
(48744,) | ir_21
(48744,) | ir_27
(48744,) | ir_33
(48744,) | ir_39
